In [1]:
import dask.dataframe as dd
import pandas as pd
from dask_ml.xgboost import XGBRegressor
from dask_ml.preprocessing import DummyEncoder
from sklearn.model_selection import train_test_split
import pathlib

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/dask_xgboost/__init__.py:7: UserWarning: Dask-XGBoost has been deprecated and is no longer maintained. The functionality of this project has been included directly in XGBoost. To use Dask and XGBoost together, please use ``xgboost.dask`` instead https://xgboost.readthedocs.io/en/latest/tutorials/dask.html.
  warnings.warn(


In [2]:
path = pathlib.Path("./Data/Filtered/")
filepaths = list(path.iterdir())

In [3]:
df = dd.read_csv(filepaths, dtype={'engine_displacement': 'object',
       'horsepower': 'object'})
df.head()

,Unnamed: 0,vin,body_type,city,city_fuel_economy,daysonmarket,dealer_zip,engine_cylinders,engine_displacement,exterior_color,...,price,salvage,seller_rating,sp_name,theft_title,transmission,transmission_display,wheel_system,wheel_system_display,year
0,0,1FM5K8HC4LGA13788,SUV / Crossover,Grandville,18.0,406,49418,V6,3000.0,Agate Black,...,59677.0,False,4.466667,Borgman Ford Mazda,False,A,Automatic,AWD,All-Wheel Drive,2020
1,1,1C4RJEBT9DC539441,SUV / Crossover,Raleigh,False,48,27616,False,5700.0,Blue,...,13500.0,False,3.555556,Raleigh Pre-Owned,False,A,Automatic,4X2,4X2,2013
2,2,JTMBK31V475028962,SUV / Crossover,Indianapolis,19.0,26,46222,V6,3500.0,White,...,7900.0,False,2.333333,California Auto Sales,False,A,5-Speed Automatic,4WD,Four-Wheel Drive,2007
3,3,4T1G11AKXLU335390,Sedan,Durham,28.0,235,27707,I4,2500.0,Super White,...,27324.0,False,4.312500,Mark Jacobson Toyota,False,A,8-Speed Automatic,FWD,Front-Wheel Drive,2020
4,4,JH4KB16565C007356,Sedan,Raleigh,16.0,36,27604,V6,3500.0,GOLD,...,5900.0,False,4.571429,Brother Auto Sales,False,A,Automatic,AWD,All-Wheel Drive,2005


In [4]:
new_df = df.drop(columns=['Unnamed: 0'])
new_df.head()

,vin,body_type,city,city_fuel_economy,daysonmarket,dealer_zip,engine_cylinders,engine_displacement,exterior_color,frame_damaged,...,price,salvage,seller_rating,sp_name,theft_title,transmission,transmission_display,wheel_system,wheel_system_display,year
0,1FM5K8HC4LGA13788,SUV / Crossover,Grandville,18.0,406,49418,V6,3000.0,Agate Black,False,...,59677.0,False,4.466667,Borgman Ford Mazda,False,A,Automatic,AWD,All-Wheel Drive,2020
1,1C4RJEBT9DC539441,SUV / Crossover,Raleigh,False,48,27616,False,5700.0,Blue,False,...,13500.0,False,3.555556,Raleigh Pre-Owned,False,A,Automatic,4X2,4X2,2013
2,JTMBK31V475028962,SUV / Crossover,Indianapolis,19.0,26,46222,V6,3500.0,White,False,...,7900.0,False,2.333333,California Auto Sales,False,A,5-Speed Automatic,4WD,Four-Wheel Drive,2007
3,4T1G11AKXLU335390,Sedan,Durham,28.0,235,27707,I4,2500.0,Super White,False,...,27324.0,False,4.312500,Mark Jacobson Toyota,False,A,8-Speed Automatic,FWD,Front-Wheel Drive,2020
4,JH4KB16565C007356,Sedan,Raleigh,16.0,36,27604,V6,3500.0,GOLD,False,...,5900.0,False,4.571429,Brother Auto Sales,False,A,Automatic,AWD,All-Wheel Drive,2005


In [114]:
test = new_df.head(10000, npartitions=10)
my_df = pd.DataFrame(test)

In [73]:
my_df.head()

,vin,body_type,city,city_fuel_economy,daysonmarket,dealer_zip,engine_cylinders,engine_displacement,exterior_color,frame_damaged,...,price,salvage,seller_rating,sp_name,theft_title,transmission,transmission_display,wheel_system,wheel_system_display,year
0,1FM5K8HC4LGA13788,SUV / Crossover,Grandville,18.0,406,49418,V6,3000.0,Agate Black,False,...,59677.0,False,4.466667,Borgman Ford Mazda,False,A,Automatic,AWD,All-Wheel Drive,2020
1,1C4RJEBT9DC539441,SUV / Crossover,Raleigh,False,48,27616,False,5700.0,Blue,False,...,13500.0,False,3.555556,Raleigh Pre-Owned,False,A,Automatic,4X2,4X2,2013
2,JTMBK31V475028962,SUV / Crossover,Indianapolis,19.0,26,46222,V6,3500.0,White,False,...,7900.0,False,2.333333,California Auto Sales,False,A,5-Speed Automatic,4WD,Four-Wheel Drive,2007
3,4T1G11AKXLU335390,Sedan,Durham,28.0,235,27707,I4,2500.0,Super White,False,...,27324.0,False,4.312500,Mark Jacobson Toyota,False,A,8-Speed Automatic,FWD,Front-Wheel Drive,2020
4,JH4KB16565C007356,Sedan,Raleigh,16.0,36,27604,V6,3500.0,GOLD,False,...,5900.0,False,4.571429,Brother Auto Sales,False,A,Automatic,AWD,All-Wheel Drive,2005


In [74]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from pprint import pprint

In [75]:
categorical_features = ['body_type','engine_cylinders', 'frame_damaged', 'salvage', 'theft_title',
                        'transmission', 'wheel_system', 'has_accidents', 'fuel_type', 'is_new']

unused_features = ['sp_name', 'wheel_system_display', 'transmission_display', 'vin', 'city', 'dealer_zip', 'daysonmarket',
                   'longitude', 'latitude', 'franchise_dealer', 'franchise_make', 'isCab', 'listed_date', 'fuel_tank_volume']

columns_that_need_fixing = ['highway_fuel_economy', 'city_fuel_economy', 'maximum_seating', 'interior_color', 'listing_color',
                            'exterior_color', 'model_name', 'make_name']

In [124]:
cols = ['price', 'year', 'make_name', 'model_name', 'frame_damaged', 'salvage', 'theft_title',
                        'transmission', 'wheel_system', 'has_accidents']
classification_df = my_df[cols]

In [125]:
dummy_classification_df = pd.get_dummies(classification_df, columns=['make_name', 'model_name', 'frame_damaged', 'salvage', 'theft_title',
                        'transmission', 'wheel_system', 'has_accidents'])

In [126]:
X = dummy_classification_df.drop('price', axis=1)
y = dummy_classification_df.price

In [127]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [128]:
clf = RandomForestClassifier(n_estimators=500, random_state=1, max_depth=2)

In [129]:
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=2, n_estimators=500, random_state=1)

In [105]:
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

In [130]:
predict = clf.predict(X_test)

In [131]:
print(f'Accuracy Score: {accuracy_score(y_test, predict)}')

Accuracy Score: 0.0348
